## Pandas连接MySQL数据库

In [ ]:
import pymysql
import pandas as pd
import numpy as np
# 打开数据库连接
dbcon = pymysql.connect(host='10.0.28.20',user='root',passwd='root96335',charset='UTF8',db='gdata')
sql = 'select * from whdtsb_201903'
df2 = pd.read_sql(sql, dbcon)
# 关闭数据库连接
dbcon.close()

# 用map+匿名格式化函数，把小数格式化为带 % 号的字符串，可以导出为字符串，但是不利于进一步分析数据，丢失了部分小数位还要再做一次转换回浮点数
df2['满意率'] = (df2['满意']/df2['评价数']).map(lambda x:format(x,'.2%'))
df2['城网工单处理及时率'] = (df2['城网未超时工单数']/df2['工单总数']).map(lambda x:format(x,'.2%'))
df2

# 只是在Dataframe显示时格式化为百分比，但是导出后还是浮点数格式
df2['满意率'] = (df2['满意']/df2['评价数'])
df2 = df2.style.format({ '满意率': '{:,.2%}'.format})
df2

`DataFrame`保存为Excel文件，默认的文件保存位置和ipynb文件在同一个路径下：

In [61]:
df2.to_excel('whdtsb_201903.xlsx',sheet_name='201903透视表汇总')

## 从数据库中导出明细并按分公司分割为多个Excel文件

In [165]:
import pymysql
import pandas as pd
import numpy as np
# 打开数据库连接
dbcon = pymysql.connect(host='10.0.28.20',user='root',passwd='root96335',charset='UTF8',db='gdata')
sql = 'select * from 维护工单抽取'
df_whgd = pd.read_sql(sql, dbcon)
# 拆分后的文件放置位置，路径的最后一个字符必须是空格？
result_path =  r'C:\Users\12112\Documents\mysql维护工单抽取明细拆分\ '
# 获取拆分条件：所属区域(分公司),去重
area_list = df_whgd['所属区域'].unique()
# 按条件拆分后分别保存新的Excel文件
for area in area_list:
# 按所属区域选择多行
    child_wb = df_whgd[df_whgd['所属区域'] == area]
    child_wb.loc[:,['片区']] = child_wb.loc[:,['片区']] .fillna(value='')
    pqarea = child_wb['片区'].unique()
    child_wb.to_excel(result_path + pqarea[0] +'片区_'+ area + '.xlsx',sheet_name='工单明细',index = False)
print('拆分完成')
# 关闭数据库连接
dbcon.close()

C:\ProgramData\Miniconda3\lib\site-packages\pandas\core\indexing.py:190: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
C:\ProgramData\Miniconda3\lib\site-packages\ipykernel_launcher.py:16: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


拆分完成
